# GLOBAL VARIABLES 

In [43]:
import os

FILE = "Aberdeen"#.zip"
PATH = os.path.join(os.getcwd(),"datasets", FILE)
BATCH_SIZE = 16
DO_LEARN = True
LEARNING_RATE = 0.001
NUM_EPOCH = 10 
WEIGHT_DECAY = 0.0001
SAVE_MODEL=True 

# MANAGEMENT OF THE DATA 

In [44]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import glob
import random 

import os.path as osp
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data

class Face_DS(torch.utils.data.Dataset):
    
    """
    A customized data loader.
    """
    def __init__(self, root=PATH, train=False, transform=None):
        """ Intialize the dataset
        """
        
        self.filenames = []
        self.root = root
        self.train = train 

        if transform is None: 
            self.transform = transforms.ToTensor()
        else: 
            self.transform=transform
        
        #archive = zipfile.ZipFile(path, 'r')^^
        #filenames = [name for name in archive.namelist() if name.endswith('.jpg')]
        #print("The path is " + path)
        filenames = glob.glob(osp.join(self.root, '*.jpg'))
        #print("The filenames are: " + str(filenames))
        
        for fn in filenames:
            self.filenames.append(fn)
        self.len = len(self.filenames)
        
        self.training_data = self.filenames
        self.training_labels = list(range(0, self.len))


        
    # You must override __getitem__ and __len__
    def __getitem__(self, index, visualization=False):
        """ ---------------------------------------------------------------------------------------------
            An item is made up of 3 images (P, P, N) and 2 target (1, 0) specifying that the 2 first 
            images are the same and the first and the third are different 
            
            If visualize = True: the image is printed 
            BUG: if visualization = True: RunTimeError due to the returned content (??)
        ----------------------------------------------------------------------------------------------- """ 

        # Visualization Part 
        if visualization: 
            # Use the torch dataloader to iterate through the dataset
            loader = torch.utils.data.DataLoader(self, shuffle=False, num_workers=0) #, batch_size=3)  
    
            #get the image
            for i_batch, sample_batched in enumerate(loader):
                if i_batch == index:
                    image = sample_batched
                    break
            
            # show images
            print("Here is the face: ")
            plt.figure(figsize=(16,8))
            img = torchvision.utils.make_grid(image)
            npimg = img.numpy()
            plt.imshow(np.transpose(npimg, (1, 2, 0)))
 
        
        imageData = Image.open(self.training_data[index]).convert("RGB") #[0]]) # Single element: later: should be a list
        target = torch.from_numpy(np.array(self.training_labels[index]) )

        #print("The tensor representing the imageData is : " + str(self.transform(imageData)))
        #print("The tensor representing the target is : " + str(target.float()))

        return self.transform(imageData), target.float() # Return x (i.e. image through a tensor) and y (i.e. label of the image)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [45]:
########################
#         TEST         #
########################

ds = Face_DS()
ds.__getitem__(1, visualization=False)



(tensor([[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [0.3529, 0.5059, 0.7176,  ..., 0.7020, 0.7451, 0.6588],
          [0.2706, 0.3922, 0.6157,  ..., 0.5725, 0.6275, 0.5765],
          [0.1176, 0.2627, 0.5216,  ..., 0.4941, 0.5608, 0.5686]],
 
         [[0.9961, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
          [0.9961, 0.9961, 0.9961,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [0.3294, 0.4824, 0.6980,  ..., 0.6745, 0.7216, 0.6392],
          [0.2392, 0.3647, 0.5922,  ..., 0.5451, 0.6039, 0.5529],
          [0.0784, 0.2235, 0.4863,  ..., 0.4667, 0.5373, 0.5451]],
 
         [[0.9804, 0.9804, 0.9804,  ..., 1.0000, 1.0000, 1.0000],
          [0.9765, 0.9765, 0.9765,  ..., 1.0000, 1.0000, 1.0000],
          [0.9608, 0.9608, 0.9608,  ...,

# DEFINITION OF THE NEURAL NETWORK 

In [46]:
from torch import nn
from torch import optim

class Net(nn.Module):
   def __init__(self):
      super().__init__()
      
      self.conv1 = nn.Conv2d(1, 64, 7)
      self.pool1 = nn.MaxPool2d(2)
      self.conv2 = nn.Conv2d(64, 128, 5)
      self.conv3 = nn.Conv2d(128, 256, 5)
      self.linear1 = nn.Linear(2304, 512)
      
      self.linear2 = nn.Linear(512, 2)
      
   def forward(self, data):
      res = []
      for i in range(2): # Siamese nets; sharing weights
         x = data[i]
         x = self.conv1(x)
         x = F.relu(x)
         x = self.pool1(x)
         x = self.conv2(x)
         x = F.relu(x)
         x = self.conv3(x)
         x = F.relu(x)
         
         x = x.view(x.shape[0], -1)
         x = self.linear1(x)
         res.append(F.relu(x))
         
      res = torch.abs(res[1] - res[0])
      res = self.linear2(res)
      return res

# DEFINITION OF THE TRAINING FUNCTION 

In [47]:
def train(model, device, train_loader, epoch, optimizer):
   model.train() # Just set the module in training mode
   
   for batch_idx, (data, target) in enumerate(train_loader):
      for i in range(len(data)):
         data[i] = data[i].to(device)
         
      optimizer.zero_grad()
      output_positive = model(data[:2])
      output_negative = model(data[0:3:2])
      
      target = target.type(torch.LongTensor).to(device)
      target_positive = torch.squeeze(target[:,0])
      target_negative = torch.squeeze(target[:,1])
      
      loss_positive = F.cross_entropy(output_positive, target_positive)
      loss_negative = F.cross_entropy(output_negative, target_negative)
      
      loss = loss_positive + loss_negative
      loss.backward()
      
      optimizer.step()
      if batch_idx % 10 == 0:
         print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*batch_size, len(train_loader.dataset), 100. * batch_idx*batch_size / len(train_loader.dataset),
            loss.item()))


# DEFINITION OF THE TESTING FUNCTION 

In [48]:
def test(model, device, test_loader):
   model.eval()
   
   with torch.no_grad():
      accurate_labels = 0
      all_labels = 0
      loss = 0
      for batch_idx, (data, target) in enumerate(test_loader):
         for i in range(len(data)):
            data[i] = data[i].to(device)
            
         output_positive = model(data[:2])
         output_negative = model(data[0:3:2])
            
         target = target.type(torch.LongTensor).to(device)
         target_positive = torch.squeeze(target[:,0])
         target_negative = torch.squeeze(target[:,1])
            
         loss_positive = F.cross_entropy(output_positive, target_positive)
         loss_negative = F.cross_entropy(output_negative, target_negative)
            
         loss = loss + loss_positive + loss_negative
            
         accurate_labels_positive = torch.sum(torch.argmax(output_positive, dim=1) == target_positive).cpu()
         accurate_labels_negative = torch.sum(torch.argmax(output_negative, dim=1) == target_negative).cpu()
            
         accurate_labels = accurate_labels + accurate_labels_positive + accurate_labels_negative
         all_labels = all_labels + len(target_positive) + len(target_negative)
      
      accuracy = 100. * accurate_labels / all_labels
      print('Test accuracy: {}/{} ({:.3f}%)\tLoss: {:.6f}'.format(accurate_labels, all_labels, accuracy, loss))
   

# DEFINITION OF THE ONE-SHOT FUNCTION 

In [49]:
def oneshot(model, device, data):
   model.eval()

   with torch.no_grad():
      for i in range(len(data)):
            data[i] = data[i].to(device)
      
      output = model(data)
      return torch.squeeze(torch.argmax(output, dim=1)).cpu().item()

# MAIN FUNCTION 

In [50]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # Specifies where the torch.tensor is allocated
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
name_model = "siamese_"
extension_model = ".pt"
   
model = Net().to(device)
   
##################
#  training mode
##################
train_loader = torch.utils.data.DataLoader(Face_DS(train=True, transform=trans), batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(Face_DS(train=False, transform=trans), batch_size=BATCH_SIZE, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
 

In [51]:

for epoch in range(NUM_EPOCH):
            
    #train(model, device, train_loader, epoch, optimizer)
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        for i in range(len(data)):
            data[i] = data[i].to(device)
        
        optimizer.zero_grad()
        output_positive = model(data[:2])
        output_negative = model(data[0:3:2])
      
        target = target.type(torch.LongTensor).to(device)
        target_positive = torch.squeeze(target[:,0])
        target_negative = torch.squeeze(target[:,1])

        loss_positive = F.cross_entropy(output_positive, target_positive)
        loss_negative = F.cross_entropy(output_negative, target_negative)
      
        loss = loss_positive + loss_negative
        loss.backward()
      
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx*batch_size, len(train_loader.dataset), 100. * batch_idx*batch_size / len(train_loader.dataset),
            loss.item()))
            

    test(model, device, test_loader)
    
if SAVE_MODEL:
    torch.save(model, (name_model + '{:03}' + extension_model).format(epoch))
    print("Model is saved!")

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 528 and 496 in dimension 2 at /Users/soumith/code/builder/wheel/pytorch-src/aten/src/TH/generic/THTensorMath.cpp:3616

In [ ]:
# OBSERVATION
# nn.linear(512) Got 528 and 496 (+ 32 of difference) 
# nn.linear(496) Got 512 and 528